In [1]:
import sys
print(sys.executable)

C:\Users\Admin\Jupyter\Posture Detection System\posture_env\Scripts\python.exe


In [2]:
# !pip install scikit-learn pandas 
# !pip install mediapipe opencv-python
#!pip install pandas

### Setup Mediapipe

In [3]:
import mediapipe as mp
import cv2
print("Mediapipe is working! Version:", mp.__version__)
print("Computer Vision is working! Version:", cv2.__version__)

Mediapipe is working! Version: 0.10.21
Computer Vision is working! Version: 4.11.0


In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

### Get Realtime Webcam Feed

In [5]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret,frame = cap.read()
    if not ret: 
        break
        
    cv2.imshow("Raw Webcam Feed",frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

### Make Detections from Feed

In [6]:
cap = cv2.VideoCapture(0)

# Initialize pose model
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make detections
    results = pose.process(image)

    # Recolor back to BGR for rendering 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw pose landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow('Pose Detection', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [7]:
results.pose_landmarks #.landmark[0].visibility

landmark {
  x: 0.659384191
  y: 0.756188273
  z: -1.32107699
  visibility: 0.998239338
}
landmark {
  x: 0.684782684
  y: 0.684061348
  z: -1.24277568
  visibility: 0.998261273
}
landmark {
  x: 0.704984188
  y: 0.68438977
  z: -1.24300969
  visibility: 0.998373926
}
landmark {
  x: 0.72554636
  y: 0.685889244
  z: -1.24295127
  visibility: 0.997763216
}
landmark {
  x: 0.616320431
  y: 0.687865376
  z: -1.27237964
  visibility: 0.998512506
}
landmark {
  x: 0.590029
  y: 0.690259278
  z: -1.27238929
  visibility: 0.998784542
}
landmark {
  x: 0.566721082
  y: 0.693644166
  z: -1.27285957
  visibility: 0.998657346
}
landmark {
  x: 0.751153588
  y: 0.724319696
  z: -0.731909811
  visibility: 0.998115242
}
landmark {
  x: 0.524809361
  y: 0.735441327
  z: -0.862461686
  visibility: 0.999067903
}
landmark {
  x: 0.693058252
  y: 0.834182501
  z: -1.12037659
  visibility: 0.995759726
}
landmark {
  x: 0.617361307
  y: 0.834701
  z: -1.15914321
  visibility: 0.997551262
}
landmark {
  x: 

# Capture Landmarks & Export to CSV
<!-- <img src="https://i.imgur.com/8bForKY.png"> -->
<!-- <img src="https://i.imgur.com/AzKNp7A.png"> -->

In [8]:
import csv
import os
import numpy as np
import time

In [9]:
num_coords = len(results.pose_landmarks.landmark)
num_coords

33

In [10]:
landmarks =["class"]
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [11]:
landmarks[-1]

'v33'

### Writing to CSV 

In [12]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

### Capturing Landmarks for classes

In [13]:
#class_name = "Upright"  #"Upright"," "Slouching," and "Leaning Forward"

In [14]:
cap = cv2.VideoCapture(0)

# Initialize pose model
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Setup defaults
recording = False
current_label = "Upright"  # Default label
print("Keys: 1=Upright  2=Slouching  3=Leaning Forward  r=toggle record  q=quit")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make detections
    results = pose.process(image)

    # Recolor back to BGR for rendering 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw pose landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Save coordinates if recording
        if recording:
            landmarks = results.pose_landmarks.landmark
            pose_row = list(np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in landmarks]).flatten())
            row = [current_label] + pose_row
            with open("coords.csv", "a", newline="") as f:  # append mode
                csv.writer(f).writerow(row)

    # Show label and recording status
    status_text = f"Label: {current_label} | Recording: {recording}"
    cv2.putText(image, status_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display feed
    cv2.imshow('Pose Detection', image)

    # Key controls
    key = cv2.waitKey(1) & 0xFF
    if key == ord('1'):
        current_label = "Good"
    elif key == ord('2'):
        current_label = "Slouch"
    elif key == ord('3'):
        current_label = "Lean"
    elif key == ord('r'):
        recording = not recording
        print(f"Recording: {recording}, Label: {current_label}, Time: {time.time()}")
    elif key == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()


Keys: 1=Upright  2=Slouching  3=Leaning Forward  r=toggle record  q=quit


# Train Custom Model Using Scikit Learn

### Read in Collected Data and Process

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [16]:
df = pd.read_csv("coords.csv")

In [30]:
df[df["class"]=="Lean"]

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
1250,Lean,0.524124,0.651781,-1.110669,0.999677,0.563843,0.563767,-1.034173,0.999484,0.588244,...,0.016757,0.000025,0.560680,3.735661,-0.423750,0.000301,0.333309,3.699188,-0.783014,0.000204
1251,Lean,0.522455,0.648207,-1.133106,0.999706,0.562933,0.561191,-1.055919,0.999528,0.587961,...,0.063496,0.000024,0.569056,3.736964,-0.379204,0.000292,0.342802,3.700468,-0.745394,0.000195
1252,Lean,0.524059,0.647120,-1.150019,0.999716,0.563906,0.561020,-1.073434,0.999542,0.588406,...,0.068532,0.000024,0.570445,3.730612,-0.359773,0.000289,0.348815,3.686675,-0.740744,0.000192
1253,Lean,0.523383,0.643807,-1.177630,0.999729,0.562876,0.557014,-1.086480,0.999561,0.587391,...,0.095341,0.000024,0.572101,3.732194,-0.335517,0.000289,0.354780,3.688869,-0.713662,0.000189
1254,Lean,0.522768,0.639855,-1.178168,0.999749,0.563172,0.553140,-1.087058,0.999593,0.587847,...,0.095710,0.000022,0.551559,3.733732,-0.347527,0.000279,0.331260,3.690854,-0.709584,0.000178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1963,Lean,0.515752,0.725412,-1.759094,0.998845,0.570064,0.616558,-1.673389,0.998171,0.600098,...,0.317753,0.000032,0.562189,4.272073,-0.150846,0.000273,0.314212,4.204051,-0.658911,0.000337
1964,Lean,0.521711,0.726006,-1.762051,0.998866,0.572594,0.617626,-1.676699,0.998226,0.601705,...,0.349017,0.000034,0.562071,4.272477,-0.112812,0.000304,0.313307,4.205026,-0.631278,0.000359
1965,Lean,0.523299,0.730146,-1.767139,0.998864,0.573836,0.620575,-1.678168,0.998211,0.603068,...,0.196695,0.000035,0.559143,4.272666,-0.287253,0.000324,0.301636,4.205544,-0.756654,0.000374
1966,Lean,0.524350,0.732729,-1.754812,0.998856,0.574705,0.622652,-1.663509,0.998187,0.603995,...,0.144065,0.000036,0.556685,4.272175,-0.346535,0.000344,0.294942,4.206175,-0.810946,0.000388


In [31]:
X = df.drop("class", axis=1) # features
y = df["class"] # target variable

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1234)

In [33]:
X_test

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
1417,0.371456,0.617374,-1.757083,0.997135,0.419329,0.518677,-1.630306,0.997692,0.446591,0.520931,...,0.188169,0.000545,0.498944,3.947963,-0.625556,0.001275,0.233962,3.907258,-0.635549,0.001708
797,0.581677,0.574047,-1.134599,0.989913,0.618297,0.513318,-1.038297,0.992021,0.639437,0.517811,...,0.106488,0.000541,0.713838,3.589974,-0.422566,0.001620,0.492528,3.564977,-0.669131,0.002352
908,0.542638,0.664329,-1.208513,0.999269,0.584000,0.581827,-1.130892,0.999268,0.606392,0.582873,...,-0.003338,0.000102,0.682964,3.666765,-0.437597,0.000586,0.462808,3.614192,-0.819904,0.000674
815,0.565749,0.582811,-1.169218,0.995906,0.610624,0.515116,-1.070666,0.996336,0.631871,0.520478,...,0.057006,0.000715,0.644122,3.555044,-0.345282,0.001604,0.422437,3.518536,-0.742837,0.002808
1443,0.618095,0.505971,-1.571681,0.994166,0.656241,0.420082,-1.448036,0.994906,0.683237,0.424182,...,0.110866,0.000400,0.721924,4.170913,-0.454935,0.001190,0.451435,4.140180,-0.875075,0.001823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,0.659892,0.427693,-0.852681,0.999789,0.676182,0.355279,-0.771222,0.999605,0.694199,0.356496,...,0.519581,0.000066,0.661170,3.473168,0.058050,0.000320,0.446005,3.448584,-0.172231,0.000170
474,0.604245,0.453932,-0.983433,0.999947,0.636003,0.388004,-0.908484,0.999872,0.655632,0.389252,...,0.490476,0.000053,0.653839,3.493285,-0.060920,0.000169,0.452299,3.477266,-0.226870,0.000124
537,0.554682,0.607414,-1.416778,0.999477,0.586769,0.528245,-1.324688,0.999355,0.607399,0.527346,...,-0.030912,0.000090,0.730918,3.495534,-0.401419,0.000384,0.523169,3.441888,-0.849231,0.000490
1581,0.892141,0.606116,-1.255795,0.999227,0.938281,0.533082,-1.198333,0.998473,0.961159,0.545805,...,1.159155,0.000180,0.754097,4.139955,0.122221,0.000246,0.469277,4.082491,0.232115,0.000402


### Train Machine Learning Classification Model

In [34]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [35]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [36]:
list(pipelines.values())[1]

,steps,"[('standardscaler', ...), ('ridgeclassifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,alpha,1.0
,fit_intercept,True
,copy_X,True
,max_iter,None


In [37]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model =  pipeline.fit(X_train,y_train)
    fit_models[algo] = model

In [38]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [39]:
fit_models["rc"].predict(X_test)

array(['Lean', 'Slouch', 'Slouch', 'Slouch', 'Lean', 'Slouch', 'Lean',
       'Slouch', 'Lean', 'Upright', 'Lean', 'Upright', 'Slouch',
       'Upright', 'Upright', 'Lean', 'Slouch', 'Lean', 'Upright',
       'Upright', 'Lean', 'Upright', 'Slouch', 'Lean', 'Slouch',
       'Upright', 'Upright', 'Slouch', 'Slouch', 'Upright', 'Slouch',
       'Slouch', 'Lean', 'Slouch', 'Slouch', 'Lean', 'Upright', 'Upright',
       'Upright', 'Lean', 'Lean', 'Upright', 'Slouch', 'Lean', 'Upright',
       'Upright', 'Slouch', 'Lean', 'Slouch', 'Slouch', 'Lean', 'Upright',
       'Upright', 'Slouch', 'Slouch', 'Lean', 'Upright', 'Slouch',
       'Slouch', 'Slouch', 'Slouch', 'Lean', 'Slouch', 'Upright',
       'Upright', 'Lean', 'Lean', 'Lean', 'Slouch', 'Upright', 'Slouch',
       'Upright', 'Slouch', 'Lean', 'Slouch', 'Lean', 'Slouch', 'Slouch',
       'Slouch', 'Lean', 'Slouch', 'Lean', 'Slouch', 'Slouch', 'Lean',
       'Slouch', 'Upright', 'Slouch', 'Slouch', 'Upright', 'Slouch',
       'Lean', 'Upr

### Evaluate and Serialize Model

In [40]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [41]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 0.9984917043740573
gb 0.9984917043740573


In [42]:
with open('posture_detect_model.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# Make Detections with Model

In [43]:
with open('posture_detect_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [44]:
model

,steps,"[('standardscaler', ...), ('randomforestclassifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2


In [50]:
import warnings
warnings.filterwarnings('ignore', message='X does not have valid feature names')

In [51]:
cap = cv2.VideoCapture(0)

# Initialize pose model
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Recolor image to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # Make detections
    results = pose.process(image)

    # Recolor back to BGR for rendering 
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Draw pose landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    try:
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in landmarks]).flatten())
        else:
            continue 
        # Predict posture
        X = pd.DataFrame([pose_row], columns=X_train.columns)
        body_language_class = model.predict(X)[0]
        body_language_prob = model.predict_proba(X)[0]

        # Get ear coordinates (change from holistic → pose)
        coords = tuple(np.multiply(
            np.array(
                (results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].x,
                 results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_EAR].y)
            ),
            [640, 480]
        ).astype(int))
    
        # Draw rectangle near ear
        cv2.rectangle(image,
                      (coords[0], coords[1]+5),
                      (coords[0]+len(body_language_class)*20, coords[1]-30),
                      (245, 117, 16), -1)
        cv2.putText(image, body_language_class, coords,
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
        # Top-left info box
        cv2.rectangle(image, (0, 0), (250, 60), (245, 117, 16), -1)
        cv2.putText(image, 'CLASS', (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, body_language_class.split(' ')[0], (90, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
        cv2.putText(image, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)], 2)),
                    (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    except Exception as e:
        print("Error during detection:", e)

    # Show video feed
    cv2.imshow('Posture Detection', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()